## Monitor transactions using anomaly detection model.     
---
**NOTE**: 

In real life scenarios financial transaction are dynamically evolving graphs. Performing anomaly detection inference on graph embeddings in live Transaction Monitoring Systems will require to update the graph and node representations after new transactions arrive. Recomputing entire graph for every newly arrived transaction will lead to unaxeptable delayes and even monitoring system failure. This problem  will be more sever if large amount of updates happen in a short time window.

Contact us at Logical Clocks and we will help you to setup end to end graph based deep anomaly detection for live Transaction Monitoring Systems. 

---

In [1]:
import json
import numpy as np

![Image7-Monitor.png](./images/model_registry.gif)

# Query Model Repository for best anomaly detection model

In [2]:
import hsml

conn = hsml.connection()
mr = conn.get_model_registry()

Connected. Call `.close()` to terminate connection gracefully.


In [3]:
MODEL_NAME="ganAml"
EVALUATION_METRIC="loss"

In [4]:
best_model = mr.get_best_model(MODEL_NAME, EVALUATION_METRIC, "min")

In [5]:
print('Model name: ' + best_model.name)
print('Model version: ' + str(best_model.version))
print(best_model.training_metrics)

Model name: ganAml
Model version: 1
{'loss': '0.768'}


### Create the deployment
Here, we fetch the model we want from the model registry and define a configuration for the deployment. For the configuration, we need to specify the serving type (default or KFserving) and in this case, since we use default serving and an sklearn model, we need to give the location of the prediction script.

In [6]:
import hsml

conn = hsml.connection()
mr = conn.get_model_registry()

Connected. Call `.close()` to terminate connection gracefully.


In [7]:
# Use the model name from notebook 4, where we registered the model
model = mr.get_model(best_model.name, version=best_model.version)


In [8]:
# Create serving instance
SERVING_NAME = "ganTfServing"

# Give it any name you want
model.deploy(name=SERVING_NAME, model_server="TENSORFLOW_SERVING", serving_tool="DEFAULT")

Deployment started, explore it at https://hopsworks0.logicalclocks.com/p/119/deployments/1


Deployment(name: 'ganTfServing')

## Fetch model server object 

In [9]:
ms = conn.get_model_serving()
deployment = ms.get_deployment('ganTfServing')

### Check Model Serving for active deployments
![Image7-Monitor.png](./images/deployment.gif)

In [10]:
# Get serving status
deployment.get_state()

PredictorState(status: 'Stopped')

In [11]:
# Start deployment
deployment.start()

  0%|          | 0/1 [00:00<?, ?it/s]

# Retrieve serving vectors and send prediction requests to the served model using Hopsworks REST API

In [12]:
import hsfs
# Create a connection
connection = hsfs.connection()
# Get the feature store handle for the project's feature store
fs = connection.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.


In [13]:
feature_view_non_sar = fs.get_feature_view('non_sar_transactions_view', 1)
feature_view_sar = fs.get_feature_view('sar_transactions_view', 1)

#### Retrieve test feature vector

In [49]:
feature_view_non_sar.get_feature_vector({'id': "0016359b"})

[1,
 0.13333333333333333,
 0.10145738951559842,
 0.424836226818694,
 0.5523306125312814,
 0.3333333333333333,
 0.4982684096194535,
 0.7698600424453773,
 0.217727110291117,
 [51.6157112121582,
  46.86443328857422,
  47.00802230834961,
  47.32992935180664,
  46.863372802734375,
  42.14142608642578,
  34.83536148071289,
  -0.0,
  38.38047409057617,
  36.33411407470703,
  52.19700241088867,
  48.818546295166016,
  47.00347900390625,
  41.64397430419922,
  43.1258544921875,
  46.23503875732422,
  48.12347412109375,
  42.12324142456055,
  49.05451202392578,
  -0.0,
  44.95339584350586,
  44.95967483520508,
  -0.0,
  47.70062255859375,
  -0.0,
  45.151161193847656,
  40.933101654052734,
  50.38645553588867,
  54.1074104309082,
  44.93682861328125,
  37.081214904785156,
  46.127140045166016]]

In [46]:
ids_to_score = ["0016359b", 
                "001dcc27", 
                "0054a022", 
                "00d6b609", 
                "00e14860", 
                "00e39a1b", 
                "014ed5cb", 
                "01ce3306", 
                "01fa19ae", 
                "01fa1d01", 
                "036dce03", 
                "03e09be4", 
                "04b23f4b"]

In [78]:
def flat2gen(alist):
    for item in alist:
        if isinstance(item, list):
            for subitem in item: yield subitem
        else:
            yield item

def model_server(input):
    data = {"inputs": [input]}
    return deployment.predict(data)

for node_id in ids_to_score:
    serving_vector = np.array(list(flat2gen(feature_view_non_sar.get_feature_vector({'id': node_id})))).reshape(1,41).tolist()
    print(model_server(serving_vector))

{'outputs': nan}
{'outputs': nan}
{'outputs': nan}
{'outputs': nan}
{'outputs': nan}
{'outputs': nan}
{'outputs': nan}
{'outputs': nan}
{'outputs': nan}
{'outputs': nan}
{'outputs': nan}
{'outputs': nan}
{'outputs': nan}
